In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8259854440397193227, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 16234276982927257465
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:83:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 6561445724530361975
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1", name: "/device:GPU:2"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 2241737499726204771
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:87:00.0, compute capability: 6.1", name: "/device:GPU:3"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 12256879186734472015
 physical_devi

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time

import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.4.0


In [5]:
output_classes = 2
batch_size = 64
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 50

nb_train_samples = 103104
nb_validation_samples = 12288
nb_test_samples = 697

In [ ]:
train_dir = 'data/train/'
validation_dir = 'data/validation'
test_dir = 'data/test'

In [ ]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 103104 images belonging to 2 classes.
Found 12288 images belonging to 2 classes.
Found 697 images belonging to 2 classes.


In [ ]:
def vgg19_model():
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

    model = Model(inputs=base_model.input,outputs=prediction)
    return model

model = vgg19_model()

In [ ]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
#          'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
          'epsilon': hp.choice('epsilon',[None,1e-08,1e-06,1e-04,1e-8,1e-9,1e-10,1e-11]),
        }

In [ ]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

#     random_seed = np.random.seed(1132)

#     train_datagen = ImageDataGenerator(
#         rescale=1. / 255,
#         featurewise_center=True,
#         featurewise_std_normalization=True)

#     test_datagen = ImageDataGenerator(rescale=1. / 255)

#     train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     validation_generator = train_datagen.flow_from_directory(
#         validation_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = False,
#         class_mode='categorical')

#     adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd_opt=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
    rmsprop_opt=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=params['epsilon'], decay=0.0)

    model.compile(optimizer = rmsprop_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1)
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    
    th = 0.3
    acc = accuracy_score(truth,predict_class > th)
    print('Accuracy:  {:.4f}'.format(acc*100))
    
    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'epsilon': 1e-08, 'lr': 1e-06, 'rho': 0.7}
Epoch 1/15
1611/1611 [==============================] - 1252s 777ms/step - loss: 0.6944 - acc: 0.7707 - val_loss: 0.4420 - val_acc: 0.8974
Epoch 2/15
1611/1611 [==============================] - 1247s 774ms/step - loss: 0.4343 - acc: 0.9010 - val_loss: 0.3671 - val_acc: 0.9239
Epoch 3/15
1611/1611 [==============================] - 1252s 777ms/step - loss: 0.3544 - acc: 0.9305 - val_loss: 0.3235 - val_acc: 0.9412
Epoch 4/15
1611/1611 [==============================] - 1240s 770ms/step - loss: 0.3075 - acc: 0.9477 - val_loss: 0.3499 - val_acc: 0.9349
Epoch 5/15
1611/1611 [==============================] - 1258s 781ms/step - loss: 0.2734 - acc: 0.9601 - val_loss: 0.2972 - val_acc: 0.9471
Epoch 6/15
1611/1611 [==============================] - 1241s 770ms/step - loss: 0.2489 - acc: 0.9686 - val_loss: 0.2772 - val_acc: 0.9573
Epoch 7/15
1611/1611 [==============================] - 1268s 787ms/step - loss: 0.2277 - acc: 0.9771

1611/1611 [==============================] - 1263s 784ms/step - loss: 0.1598 - acc: 0.9441 - val_loss: 0.7821 - val_acc: 0.6862
Epoch 7/15
1611/1611 [==============================] - 1238s 769ms/step - loss: 0.1433 - acc: 0.9507 - val_loss: 0.9538 - val_acc: 0.6862
Epoch 8/15
1611/1611 [==============================] - 1242s 771ms/step - loss: 0.1355 - acc: 0.9546 - val_loss: 0.8993 - val_acc: 0.6862
Epoch 9/15
1611/1611 [==============================] - 1243s 772ms/step - loss: 0.1251 - acc: 0.9584 - val_loss: 0.8571 - val_acc: 0.6862
Epoch 10/15
1611/1611 [==============================] - 1236s 767ms/step - loss: 0.1221 - acc: 0.9607 - val_loss: 0.9635 - val_acc: 0.6862
Epoch 11/15
1611/1611 [==============================] - 1244s 772ms/step - loss: 0.1133 - acc: 0.9635 - val_loss: 0.8859 - val_acc: 0.6862
Epoch 12/15
1611/1611 [==============================] - 1240s 770ms/step - loss: 0.5247 - acc: 0.7933 - val_loss: 0.6727 - val_acc: 0.6862
Epoch 13/15
1611/1611 [============

1611/1611 [==============================] - 1248s 775ms/step - loss: 5.0600 - acc: 0.6861 - val_loss: 5.0579 - val_acc: 0.6862
Epoch 12/15
1611/1611 [==============================] - 1225s 760ms/step - loss: 5.0600 - acc: 0.6861 - val_loss: 5.0579 - val_acc: 0.6862
Epoch 13/15
1611/1611 [==============================] - 1231s 764ms/step - loss: 5.0600 - acc: 0.6861 - val_loss: 5.0579 - val_acc: 0.6862
Epoch 14/15
1611/1611 [==============================] - 1232s 765ms/step - loss: 5.0600 - acc: 0.6861 - val_loss: 5.0579 - val_acc: 0.6862
Epoch 15/15
1611/1611 [==============================] - 1221s 758ms/step - loss: 5.0600 - acc: 0.6861 - val_loss: 5.0579 - val_acc: 0.6862
Validation Score:  4.986535739898682
Validation Accuracy:  0.690625
11/10 [==============================] - 10s 915ms/step
No of errors = 216/697
Accuracy:  69.0100
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_